# How Time Works

At the beginning `t=0`

In [1]:
import sys
import pandas as pd
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
# import helpers.functions as f
import ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [2]:
time = {
    "label": "time",
    "objid":  f.uuid(n=13),
    "name": "time",
    "originTime":0,
}
time

{'label': 'time', 'objid': '2576061793482', 'name': 'time', 'originTime': 0}

Time has no edges. It sits in it's own record. 

**Note** The cell below will reset time. This will cause disruptions in the larger system. 

In [6]:
# data = {"nodes":[time],'edges':[]}
# db.upload_data(data,verbose=True)

In [9]:
res = db.run_query("g.V().hasLabel('time').valueMap()")
time = [db.clean_node(n) for n in res]

In [11]:
time[0]

{'objid': '2576061793482',
 'name': 'time',
 'originTime': 0,
 'username': 'notebook',
 'objtype': 'time',
 'id': '2576061793482'}